In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from tqdm import tqdm

# Problem 1

In [ ]:
def importance_sample(lambd, n_samples: list):
    proposal = scipy.stats.norm(0, 1/lambd)
    target = scipy.stats.norm(0, 1)
    all_weights = []
    all_samples = []
    for N in n_samples:
        samples = proposal.rvs(size=N)
        weights = np.exp(target.logpdf(samples) - proposal.logpdf(samples))
        all_samples.append(samples)
        all_weights.append(weights)
    return all_samples, all_weights

In [ ]:
n_samples = list(range(10, 10000, 10))

In [ ]:
lambd = 1.5
all_samples, all_weights = importance_sample(lambd, n_samples)
normalizing_constants = [np.mean(weights) for weights in all_weights]
plt.plot(n_samples, normalizing_constants)

In [ ]:
lambd = 2.5
all_samples, all_weights = importance_sample(lambd, n_samples)
normalizing_constants = [np.mean(weights) for weights in all_weights]
plt.plot(n_samples, normalizing_constants)

# Problem 2

In [ ]:
# Constants
A = 0.9  # state transition matrix
Q = 0.5  # state variance
C = 1.3  # observation matrix
R = 0.1  # observation variance

## a) Simulate the model


In [ ]:
def step_x(x):
    return A * x + scipy.stats.norm(0, np.sqrt(Q)).rvs() # np.random.normal(scale=np.sqrt(Q))

def step_y(x):
    return C * x + scipy.stats.norm(0, np.sqrt(R)).rvs(x.shape[0]) # np.random.normal(scale=np.sqrt(R))

def simulate(initial_x, step_x_fcn, step_y_fcn, n_timesteps):
    xs = [initial_x] + [None] * n_timesteps
    for t in range(n_timesteps):
        xs[t+1] = step_x_fcn(xs[t])
    
    xs = np.array(xs[1:])
    ys = step_y_fcn(xs)
    return xs, ys

In [ ]:
np.random.seed(0)
T = 2000
initial_x = np.random.normal(0, 1)
x_data, y_data = simulate(initial_x, step_x, step_y, 2000)
x_data.shape

In [ ]:
plt.plot(y_data, label="$y_t$")
plt.plot(x_data, label="$x_t$")
plt.xlabel("T")
plt.legend()

In [ ]:
plt.plot(y_data[100:200], label="$y_t$")
plt.plot(x_data[100:200], label="$x_t$")
plt.xlabel("T")
plt.legend()

## b) Kalman Filtering

In [ ]:
def weighted_mean_and_var(values, weights):
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)
    return (average, variance)

In [ ]:
# Constants
P0 = 1  # initial state variance

In [ ]:
np.random.seed(0)
initial_x = 0  # scipy.stats.norm(0, P0).rvs()
initial_Pt_filtering = P0

In [ ]:
def kalman_filter(initial_x, initial_Pt_filtering, T, A, C, Q, R):
    xs = [initial_x] + [None] * T
    Pts_filtering = [initial_Pt_filtering] + [None] * T
    for t in range(T):
        Pt_predictive = A * Pts_filtering[t] * A + Q

        Kt = Pt_predictive * C / (C * Pt_predictive * C + R)

        # state update
        xs[t+1] = A * xs[t] + Kt * (y_data[t] - C * A * xs[t])

        # variance update
        Pts_filtering[t+1] = Pt_predictive - Kt * C * Pt_predictive

    xs = np.array(xs[1:])
    Pts_filtering = np.array(Pts_filtering[1:])
    return xs, Pts_filtering

In [ ]:
kalman_particles, kalman_variances = kalman_filter(initial_x, initial_Pt_filtering, T, A, C, Q, R)

In [ ]:
plt.plot(kalman_particles[200:250], label="$\hat{x}_t$")
plt.plot(x_data[200:250], label="$x_t$")
plt.xlabel("T")
plt.legend()

In [ ]:
plt.plot(kalman_particles, label="$\hat{x}_t$")
plt.plot(x_data, label="$x_t$")
plt.xlabel("T")
plt.legend()

In [ ]:
plt.plot((kalman_particles - x_data), label="$|\hat{x}_t - x_t$|")
plt.xlabel("T")
plt.legend()

In [ ]:
plt.plot(kalman_variances)

## c) Bootstrap Particle Filtering

In [ ]:
# Bootstrap Particle Filter

def bootstrap_pf(initial_particles, A, C, Q, R, seed=0):
    np.random.seed(seed)
    N = len(initial_particles)
    print(f"Running with {N} particles")
    weights = [np.array([1/N] * N)] + [None] * T
    particles = [initial_particles] + [None] * T  # draw initial particles
    mean_filtering = [None] * T
    var_filtering = [None] * T
    ancestor_indices = [None] * T

    for t in tqdm(range(T)):
        # RESAMPLE
        a_indices = np.random.choice(range(N), p=weights[t], replace=True, size=N)
        ancestor_indices[t] = a_indices

        # PROPAGATE
        # state
        fcn = A * particles[t][a_indices]
        proposal_dist = scipy.stats.norm(fcn, np.sqrt(Q))
        particles[t+1] = proposal_dist.rvs()

        # measurement
        fcn = C * particles[t+1]
        measurement_dist = scipy.stats.norm(fcn, np.sqrt(R))

        # WEIGHT
        log_weights_unnorm = measurement_dist.logpdf(y_data[t])
        weights_unnorm = np.exp(log_weights_unnorm - np.max(log_weights_unnorm))
        weights[t+1] = weights_unnorm / np.sum(weights_unnorm)

        mean_filtering[t], var_filtering[t] = weighted_mean_and_var(particles[t+1], weights[t+1])
        # mean_filtering[t] = np.sum(weights[t+1] * particles[t+1])
        # var_filtering[t] = None  # np.sum(weights[t+1] * particles[t+1])

    weights = np.array(weights[1:])
    particles = np.array(particles[1:])
    mean_filtering = np.array(mean_filtering)
    var_filtering = np.array(var_filtering)
    ancestor_indices = np.array(ancestor_indices)
    return particles, weights, mean_filtering, var_filtering, ancestor_indices

In [ ]:
N = 500
initial_particle_dist = scipy.stats.norm(0, np.sqrt(P0))
initial_particles = initial_particle_dist.rvs(N)

bpf_particles, bpf_weights, bpf_mean_filtering, bpf_var_filtering, bpf_ancestor_indices = bootstrap_pf(initial_particles, A, C, Q, R)

In [ ]:
plt.plot(bpf_mean_filtering, label="$\hat{x}_t$")
plt.plot(x_data, label="$x_t$")
plt.xlabel("T")
plt.legend()

In [ ]:
plt.plot(bpf_mean_filtering[200:250], label="$\hat{x}_t$")
plt.plot(x_data[200:250], label="$x_t$")
plt.xlabel("T")
plt.legend()

In [ ]:
plt.plot((bpf_mean_filtering - x_data), label="$|\hat{x}_t - x_t$|")
plt.xlabel("T")
plt.legend()

In [ ]:
plt.plot(bpf_var_filtering, label="Var")
plt.legend()

### Comparison to the Kalman Filter

In [ ]:
initial_particle_dist = scipy.stats.norm(0, np.sqrt(P0))

Ns = [10, 50, 100, 2000, 5000, 100000]

all_mean_filtering = []
all_var_filtering = []

for N in Ns:
    initial_particles = initial_particle_dist.rvs(N)
    particles, weights, mean_filtering, var_filtering, ancestor_indices = bootstrap_pf(initial_particles, A, C, Q, R)
    all_mean_filtering.append(mean_filtering)
    all_var_filtering.append(var_filtering)

In [ ]:
avg_absolute_differences_of_mean = [np.mean(np.abs(kalman_particles - np.array(mean_filtering))) for mean_filtering in all_mean_filtering]
avg_absolute_differences_of_var = [np.mean(np.abs(kalman_variances - np.array(var_filtering))) for var_filtering in all_var_filtering]

In [ ]:
Ns, avg_absolute_differences_of_mean

In [ ]:
Ns, avg_absolute_differences_of_var

## d) Fully Adapted Particle Filtering

In [ ]:
initial_particle_dist = scipy.stats.norm(1, 1)  # the actual best initial distribution

# Fully Adapted Particle Filter

def fully_adapted_pf(initial_particles, A, C, Q, R, seed=0):
    np.random.seed(0)
    N = len(initial_particles)
    weights = [np.array([1/N] * N)] + [None] * T  # these are nu weights
    particles = [None] * T + [initial_particles]  # draw initial particles - put at index -1
    mean_observation = [None] * T  # p(y_t|x_t)
    std_observation = [None] * T
    mean_state_prediction = [None] * T  # p(x_t|x_t-1)
    std_state_prediction = [None] * T
    mean_marginal_filtering = [None] * T  # p(x_t|x_t-1, y_t)
    std_marginal_filtering = [None] * T
    loglikelihood = 0

    for t in tqdm(range(T)):
        # WEIGHT
        # measurement
        fcn = np.cos(particles[t-1]) ** 2
        mean = C * fcn
        sigma = np.sqrt(C * Q * C + R)
        measurement_proposal = scipy.stats.norm(mean, sigma)

        # compute weights (nu)
        log_weights_unnorm = measurement_proposal.logpdf(y_data[t])
        log_weights_max = np.max(log_weights_unnorm)
        weights_unnorm = np.exp(log_weights_unnorm - log_weights_max) + log_weights_max
        weights[t] = weights_unnorm / np.sum(weights_unnorm)

        # RESAMPLE
        ancestor_indices = np.random.choice(range(N), p=weights[t], replace=True, size=N)

        # PROPAGATE
        # state
        fcn = np.cos(particles[t-1][ancestor_indices]) ** 2
        mean = fcn + K * (y_data[t] - C * fcn)
        proposal_dist = scipy.stats.norm(mean, np.sqrt(Sigma))
        particles[t] = proposal_dist.rvs()
        # measurement (optional)
        measurement_dist = scipy.stats.norm(C * np.mean(particles[t]), np.sqrt(R))
        mean_observation[t] = measurement_dist.mean()
        std_observation[t] = measurement_dist.std()

        # mean_marginal_filtering[t] = np.mean(proposal_dist.mean())  # particles incorporate y_data from same time step (hence filtering)
        mean_marginal_filtering[t] = np.mean(proposal_dist.rvs())  # particles incorporate y_data from same time step (hence filtering)
        std_marginal_filtering[t] = np.mean(proposal_dist.std())

        fcn = np.cos(particles[t-1]) ** 2  # no resampling here
        prediction_dist = scipy.stats.norm(fcn, np.sqrt(Q))  # prediction formed by ignoring y_data (not available)
        mean_state_prediction[t] = np.mean(prediction_dist.mean())
        std_state_prediction[t] = np.mean(prediction_dist.std())

        # loglikelihood += np.log(np.sum(weights_unnorm)) - np.log(N)

    weights = np.array(weights[:-1])
    particles = np.array(particles[-1:] + particles[1:-1])  # move initial particle to index 0  #  np.array(particles[:-1])
    mean_marginal_filtering = np.array(mean_marginal_filtering)
    std_marginal_filtering = np.array(std_marginal_filtering)
    mean_state_prediction = np.array(mean_state_prediction)
    std_state_prediction = np.array(std_state_prediction)
    mean_observation = np.array(mean_observation)
    std_observation = np.array(std_observation)
    loglikelihood = np.array(loglikelihood)

## e) Genealogy of Fully Adapted Particle Filtering

In [ ]:
def backtrack_genealogy(list_index, list_sample):
    aux_list_index = copy.deepcopy(list_index)
    genealogy = [list_sample[-1].reshape(1,-1)]
    
    for k in range(len(list_index)-1, 0, -1):
        index_previous = aux_list_index[k]
        aux_list_index[k-1] = aux_list_index[k-1][index_previous]
        genealogy.insert(0, list_sample[k-1][index_previous].reshape(1,-1))
  
    genealogy = np.concatenate(genealogy,axis =0)
    return genealogy

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 10))

ax.plot(list(range(T)), genealogy, marker='o', color='red')  #, linestyle='--')

for t in range(T - 1):
    p = np.array([particles[t][ancestor_indices[t+1]], particles[t+1]])
    ax.plot([t, t+1], p, marker='o', color='grey', alpha=0.5);  #, linestyle='--')

ax.plot([0, 0], [particles[0], particles[0]], marker='o', color='grey', alpha=0.5);  #, linestyle='--')

## f) Genealogy of Fully Adapted Particle Filtering with Systematic Resampling

In [ ]:
def systematic_resampling(w, x, n_strata=None):
    n_strata = len(w) if n_strata is None else n_strata
    u = (np.arange(n_strata) + np.random.rand())/n_strata
    bins = np.cumsum(w)
    return x[np.digitize(u,bins)]

## e) Genealogy of Fully Adapted Particle Filtering with Systematic and Adaptive Resampling

# Problem 3

# Problem 4

# Problem 5

# Problem 6